In [17]:
%%writefile matrix_mul_tiled.cu
#include <iostream>
#include <cuda_runtime.h>
#include <cstdlib>

#define TILE_SIZE 16  // Define tile size
#define N 1024        // Matrix size N x N (adjust this to fit GPU memory)

__global__ void matMulTiled(float* A, float* B, float* C, int n) {
    __shared__ float tileA[TILE_SIZE][TILE_SIZE];
    __shared__ float tileB[TILE_SIZE][TILE_SIZE];

    int row = blockIdx.y * TILE_SIZE + threadIdx.y;
    int col = blockIdx.x * TILE_SIZE + threadIdx.x;

    float sum = 0.0f;

    for (int i = 0; i < (n + TILE_SIZE - 1) / TILE_SIZE; i++) {
        // Load tiles into shared memory
        if (row < n && (i * TILE_SIZE + threadIdx.x) < n)
            tileA[threadIdx.y][threadIdx.x] = A[row * n + i * TILE_SIZE + threadIdx.x];
        else
            tileA[threadIdx.y][threadIdx.x] = 0.0f;

        if (col < n && (i * TILE_SIZE + threadIdx.y) < n)
            tileB[threadIdx.y][threadIdx.x] = B[(i * TILE_SIZE + threadIdx.y) * n + col];
        else
            tileB[threadIdx.y][threadIdx.x] = 0.0f;

        __syncthreads();  // Wait for all threads to finish loading

        // Multiply the two tiles
        for (int j = 0; j < TILE_SIZE; j++) {
            sum += tileA[threadIdx.y][j] * tileB[j][threadIdx.x];
        }

        __syncthreads();  // Wait for all threads to finish computing
    }

    if (row < n && col < n) {
        C[row * n + col] = sum;
    }
}

// Function to initialize a random matrix
void randomMatrix(float* mat, int n) {
    for (int i = 0; i < n * n; i++) {
        mat[i] = static_cast<float>(rand()) / RAND_MAX;
    }
}

// Function to measure execution time of kernel
void measureExecutionTime(float *d_A, float *d_B, float *d_C, dim3 gridSize, dim3 blockSize, int n) {
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);

    // Launch the kernel
    matMulTiled<<<gridSize, blockSize>>>(d_A, d_B, d_C, n);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    std::cout << "Execution Time: " << milliseconds << " ms" << std::endl;

    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

// Function to print a portion of the matrix
void printMatrix(float* mat, int n) {
    std::cout << "Resulting Matrix (first 5x5 block):" << std::endl;
    for (int i = 0; i < 5 && i < n; ++i) {
        for (int j = 0; j < 5 && j < n; ++j) {
            std::cout << mat[i * n + j] << " ";
        }
        std::cout << std::endl;
    }
}

int main() {
    int n = N;
    size_t size = n * n * sizeof(float);

    // Allocate memory on host (CPU)
    float *A, *B, *C;
    A = (float*)malloc(size);
    B = (float*)malloc(size);
    C = (float*)malloc(size);

    // Initialize matrices A and B with random values
    randomMatrix(A, n);
    randomMatrix(B, n);

    // Allocate memory on device (GPU)
    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);

    // Copy data from host to device
    cudaMemcpy(d_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, size, cudaMemcpyHostToDevice);

    // Define block size and grid size
    dim3 blockSize(TILE_SIZE, TILE_SIZE);
    dim3 gridSize((n + TILE_SIZE - 1) / TILE_SIZE, (n + TILE_SIZE - 1) / TILE_SIZE);

    std::cout << "Matrix Multiplication Using Tiling:\n";

    // Measure execution time of the tiled matrix multiplication
    measureExecutionTime(d_A, d_B, d_C, gridSize, blockSize, n);

    // Copy the result back to host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    // Print the resulting matrix
    printMatrix(C, n);

    // Free memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    free(A);
    free(B);
    free(C);

    return 0;
}


Overwriting matrix_mul_tiled.cu


In [18]:
!nvcc -o matrix_mul_tiled matrix_mul_tiled.cu



In [19]:
!./matrix_mul_tiled



Matrix Multiplication Using Tiling:
Execution Time: 5.99962 ms
Resulting Matrix (first 5x5 block):
264.159 257.258 250.823 261.542 258.473 
256.462 251.636 252.988 252.483 249.185 
261.322 257.258 249.169 251.769 247.417 
260.235 259.321 251.849 249.62 253.515 
256.806 250.774 251.65 249.034 254.128 


In [20]:
%%writefile matrix_mul.cu
#include <iostream>
#include <cuda_runtime.h>
#include <cstdlib>

#define N 1024  // Matrix size N x N (adjust to fit GPU memory)

__global__ void matMul(float* A, float* B, float* C, int n) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    float sum = 0.0f;

    if (row < n && col < n) {
        for (int i = 0; i < n; i++) {
            sum += A[row * n + i] * B[i * n + col];
        }
        C[row * n + col] = sum;
    }
}

// Function to initialize a random matrix
void randomMatrix(float* mat, int n) {
    for (int i = 0; i < n * n; i++) {
        mat[i] = static_cast<float>(rand()) / RAND_MAX;
    }
}

// Function to measure execution time of the kernel
void measureExecutionTime(float *d_A, float *d_B, float *d_C, dim3 gridSize, dim3 blockSize, int n) {
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);

    // Launch the kernel
    matMul<<<gridSize, blockSize>>>(d_A, d_B, d_C, n);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    std::cout << "Execution Time: " << milliseconds << " ms" << std::endl;

    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

// Function to print a portion of the matrix
void printMatrix(float* mat, int n) {
    std::cout << "Resulting Matrix (first 5x5 block):" << std::endl;
    for (int i = 0; i < 5 && i < n; ++i) {
        for (int j = 0; j < 5 && j < n; ++j) {
            std::cout << mat[i * n + j] << " ";
        }
        std::cout << std::endl;
    }
}

int main() {
    int n = N;
    size_t size = n * n * sizeof(float);

    // Allocate memory on host (CPU)
    float *A, *B, *C;
    A = (float*)malloc(size);
    B = (float*)malloc(size);
    C = (float*)malloc(size);

    // Initialize matrices A and B with random values
    randomMatrix(A, n);
    randomMatrix(B, n);

    // Allocate memory on device (GPU)
    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);

    // Copy data from host to device
    cudaMemcpy(d_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, size, cudaMemcpyHostToDevice);

    // Define block size and grid size
    dim3 blockSize(16, 16);
    dim3 gridSize((n + blockSize.x - 1) / blockSize.x, (n + blockSize.y - 1) / blockSize.y);

    std::cout << "Matrix Multiplication Without Tiling:\n";

    // Measure execution time of the matrix multiplication
    measureExecutionTime(d_A, d_B, d_C, gridSize, blockSize, n);

    // Copy the result back to host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    // Print the resulting matrix
    printMatrix(C, n);

    // Free memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    free(A);
    free(B);
    free(C);

    return 0;
}


Writing matrix_mul.cu


In [21]:
!nvcc -o matrix_mul matrix_mul.cu


In [22]:
!./matrix_mul


Matrix Multiplication Without Tiling:
Execution Time: 55.5 ms
Resulting Matrix (first 5x5 block):
264.159 257.258 250.823 261.542 258.473 
256.462 251.636 252.988 252.483 249.185 
261.322 257.258 249.169 251.769 247.417 
260.235 259.321 251.849 249.62 253.515 
256.806 250.774 251.65 249.034 254.128 
